In [1]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
import pandas as pd

# ------------------------------------------------
# 1) Helper Function: Parsing Time-Parameterized Vectors
# ------------------------------------------------
def parse_time_vector(vector_string, num_years):
    """
    Parses strings like:
      - "12" -> constant 12.0 for all years
      - "5 10S 12 5R 20" -> from year 1..9 => 5, year 10 => 12,
                            then ramp from 12 to 20 over 5 years, then 20% in perpetuity.
    Returns a numpy array (length = num_years) with the yearly rates.
    """
    tokens = vector_string.strip().split()
    
    # If there's only one token and it's just a number, interpret as constant
    if len(tokens) == 1:
        try:
            val = float(tokens[0])
            return np.full(num_years, val)
        except ValueError:
            raise ValueError(f"Could not parse constant rate from '{vector_string}'")
    
    try:
        current_rate = float(tokens[0])
    except ValueError:
        raise ValueError(f"Could not parse the initial rate from '{tokens[0]}'")
    
    year_rates = np.full(num_years, current_rate)
    current_year = 1  # 1-based index
    idx = 1
    
    while idx < len(tokens):
        duration_and_type = tokens[idx]
        idx += 1
        if idx >= len(tokens):
            raise ValueError(f"Unexpected end of tokens after {duration_and_type}")
        new_rate_str = tokens[idx]
        idx += 1
        
        mode = duration_and_type[-1].upper()  # 'S' or 'R'
        duration_str = duration_and_type[:-1]
        
        try:
            duration = int(duration_str)
        except ValueError:
            raise ValueError(f"Could not parse duration from '{duration_str}'")
        
        try:
            new_rate = float(new_rate_str)
        except ValueError:
            raise ValueError(f"Could not parse new rate from '{new_rate_str}'")
        
        start_year = current_year
        end_year = current_year + duration
        if end_year > num_years:
            end_year = num_years
        
        if mode == 'S':
            # Step at (start_year + duration - 1)
            for y in range(start_year, end_year):
                if y-1 < num_years:
                    year_rates[y-1] = current_rate
            step_year_index = end_year - 1
            if 0 <= step_year_index < num_years:
                year_rates[step_year_index] = new_rate
            current_rate = new_rate
            current_year = end_year
        
        elif mode == 'R':
            # Ramp linearly
            rate_diff = new_rate - current_rate
            for i_year in range(duration):
                frac = i_year / float(duration - 1) if duration > 1 else 1.0
                actual_rate = current_rate + frac * rate_diff
                actual_year = start_year + i_year
                if actual_year <= num_years:
                    year_rates[actual_year - 1] = actual_rate
            current_rate = new_rate
            current_year = end_year
        
        else:
            raise ValueError(f"Unknown mode '{mode}' in parse_time_vector.")
    
    for y in range(current_year, num_years+1):
        if y-1 < num_years:
            year_rates[y-1] = current_rate
    
    return year_rates


# ------------------------------------------------
# 2) Helper Function: Draw from Chosen Distribution
# ------------------------------------------------
def draw_random(dist_type, mean, stdev, size=1):
    """
    dist_type: 'normal' or 'lognormal'.
    mean, stdev refer to the *arithmetic* mean and std. dev (if lognormal).
    """
    dist_type = dist_type.lower()
    if dist_type == 'normal':
        return np.random.normal(loc=mean, scale=stdev, size=size)
    elif dist_type == 'lognormal':
        if mean <= 0:
            return np.zeros(size)
        sigma_sq = stdev**2
        mu_sq = mean**2
        phi = np.sqrt(np.log(1 + sigma_sq/mu_sq))   
        m   = np.log(mu_sq / np.sqrt(sigma_sq+mu_sq))
        return np.random.lognormal(mean=m, sigma=phi, size=size)
    else:
        raise ValueError(f"Unsupported distribution type '{dist_type}'")


# ------------------------------------------------
# 3) Main Simulation Function
# ------------------------------------------------
def monte_carlo_retirement(
    num_sims,
    num_years,
    # Inflation vectors
    inflation_rate_vector,           
    inflation_vol_vector,            
    inflation_dist='normal',
    
    # Asset returns
    asset_mean_vectors=None,
    asset_vol_vectors=None,
    asset_dist_types=None,  # e.g. ['normal','normal','normal',...], length=10
    
    # Allocations
    allocation_vectors=None,  # length=10
    
    # Contribution parameters
    initial_contribution=10000.0,
    contrib_growth_vector="2",
    contrib_vol_vector="1",
    contrib_dist='normal',
    peg_contribution_to_inflation=True,
    contribution_stop_year=30,   # <---- new param
    
    # Drawdown parameters
    initial_drawdown=0.0,
    drawdown_growth_vector="0",
    drawdown_vol_vector="0",
    drawdown_dist='normal',
    peg_drawdown_to_inflation=True,
    drawdown_start_year=30,      # <---- new param
    
    # Initial balance
    initial_balance=0.0,
    
    # For reproducibility
    random_seed=None
):
    """
    Returns a DataFrame with both nominal and real (inflation-adjusted) statistics by year.
    
    'contribution_stop_year': no new contributions after this year index (0-based or 1-based).
    'drawdown_start_year': no drawdowns before this year index (0-based or 1-based).
    
    NOTE: For clarity, assume these are 0-based indices if you want them to 
          exactly match the loop 't'. Or if you treat them as "Year" numbers 
          in a 1-based sense, you can do the comparisons accordingly.
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    
    # 1) Parse time-varying vectors
    inflation_mean_array = parse_time_vector(inflation_rate_vector, num_years)
    inflation_stdev_array = parse_time_vector(inflation_vol_vector, num_years)
    
    if asset_mean_vectors is None or asset_vol_vectors is None or asset_dist_types is None:
        raise ValueError("Must provide asset_mean_vectors, asset_vol_vectors, asset_dist_types (all length=10).")
    if len(asset_mean_vectors) != 10 or len(asset_vol_vectors) != 10 or len(asset_dist_types) != 10:
        raise ValueError("asset_* parameters must each be length=10.")
    
    asset_mean_arrays = [parse_time_vector(v, num_years) for v in asset_mean_vectors]
    asset_stdev_arrays = [parse_time_vector(v, num_years) for v in asset_vol_vectors]
    
    if allocation_vectors is None or len(allocation_vectors) != 10:
        raise ValueError("Must provide 10 allocation_vectors (one per asset).")
    allocation_arrays = [parse_time_vector(v, num_years) for v in allocation_vectors]
    
    contrib_growth_array = parse_time_vector(contrib_growth_vector, num_years)
    contrib_stdev_array = parse_time_vector(contrib_vol_vector, num_years)
    
    drawdown_growth_array = parse_time_vector(drawdown_growth_vector, num_years)
    drawdown_stdev_array = parse_time_vector(drawdown_vol_vector, num_years)
    
    # 2) Prepare arrays to store results
    all_balances = np.zeros((num_sims, num_years))
    all_inflations = np.zeros((num_sims, num_years))
    all_contributions = np.zeros((num_sims, num_years))
    all_drawdowns = np.zeros((num_sims, num_years))
    all_effective_returns = np.zeros((num_sims, num_years))
    
    all_real_balances = np.zeros((num_sims, num_years))
    all_real_contributions = np.zeros((num_sims, num_years))
    all_real_drawdowns = np.zeros((num_sims, num_years))
    
    all_cum_inflation = np.ones((num_sims, num_years))
    
    # 3) Run simulations
    for s in range(num_sims):
        balance = initial_balance
        # We'll track last year's nominal contribution/drawdown
        last_contribution = initial_contribution
        last_drawdown = initial_drawdown
        cum_infl = 1.0
        
        for t in range(num_years):
            # 3.1 Draw inflation
            infl_mean_t = inflation_mean_array[t] / 100.0
            infl_std_t  = inflation_stdev_array[t] / 100.0
            this_infl = draw_random(inflation_dist, infl_mean_t, infl_std_t, size=1)[0]
            cum_infl *= (1.0 + this_infl)
            all_cum_inflation[s, t] = cum_infl
            
            # 3.2 Draw returns for the 10 assets
            asset_returns_t = np.zeros(10)
            for i in range(10):
                mean_i = asset_mean_arrays[i][t] / 100.0
                std_i  = asset_stdev_arrays[i][t] / 100.0
                dist_i = asset_dist_types[i]
                asset_returns_t[i] = draw_random(dist_i, mean_i, std_i, size=1)[0]
            
            # 3.3 Contribution logic
            #     If t < contribution_stop_year, we compute a new contribution.
            #     Otherwise, it's zero.
            if t < contribution_stop_year:
                c_growth_mean_t = contrib_growth_array[t] / 100.0
                c_growth_std_t  = contrib_stdev_array[t] / 100.0
                c_growth_draw   = draw_random(contrib_dist, c_growth_mean_t, c_growth_std_t, size=1)[0]
                
                if peg_contribution_to_inflation:
                    c_growth = this_infl
                else:
                    c_growth = c_growth_draw
                
                if t == 0:
                    this_contribution = last_contribution
                else:
                    this_contribution = all_contributions[s, t-1] * (1 + c_growth)
            else:
                # After the stop year, no further contributions
                this_contribution = 0.0
            
            balance += this_contribution
            
            # 3.4 Weighted return
            weighted_return = 0.0
            for i in range(10):
                alloc_pct = allocation_arrays[i][t] / 100.0
                weighted_return += alloc_pct * asset_returns_t[i]
            balance *= (1.0 + weighted_return)
            
            # 3.5 Drawdown logic
            #     If t < drawdown_start_year, we do zero drawdowns.
            #     Otherwise, we do normal drawdown growth logic.
            if t < drawdown_start_year:
                this_drawdown = 0.0
            else:
                d_growth_mean_t = drawdown_growth_array[t] / 100.0
                d_growth_std_t  = drawdown_stdev_array[t] / 100.0
                d_growth_draw   = draw_random(drawdown_dist, d_growth_mean_t, d_growth_std_t, size=1)[0]
                
                if peg_drawdown_to_inflation:
                    d_growth = this_infl
                else:
                    d_growth = d_growth_draw
                
                if t == 0:
                    this_drawdown = last_drawdown
                else:
                    this_drawdown = all_drawdowns[s, t-1] * (1 + d_growth)
            
            balance -= this_drawdown
            if balance < 0:
                balance = 0.0
            
            # 3.6 Store results
            all_balances[s, t] = balance
            all_inflations[s, t] = this_infl
            all_contributions[s, t] = this_contribution
            all_drawdowns[s, t] = this_drawdown
            all_effective_returns[s, t] = weighted_return
            
            # Real values
            all_real_balances[s, t]       = balance / cum_infl
            all_real_contributions[s, t]  = this_contribution / cum_infl
            all_real_drawdowns[s, t]      = this_drawdown / cum_infl
    
    # 4) Summaries
    percentiles = [10, 25, 50, 75, 90]
    output_dict = {'Year': list(range(1, num_years+1))}
    
    # Nominal balance percentiles
    for p in percentiles:
        col_name = f"Balance_P{p}"
        output_dict[col_name] = [
            np.percentile(all_balances[:, t], p) for t in range(num_years)
        ]
    
    # Real balance percentiles
    for p in percentiles:
        col_name = f"Real_Balance_P{p}"
        output_dict[col_name] = [
            np.percentile(all_real_balances[:, t], p) for t in range(num_years)
        ]
    
    # Averages
    output_dict['Avg_Effective_Return'] = [
        np.mean(all_effective_returns[:, t]) * 100.0 for t in range(num_years)
    ]
    output_dict['Avg_Inflation'] = [
        np.mean(all_inflations[:, t]) * 100.0 for t in range(num_years)
    ]
    output_dict['Avg_Contribution'] = [
        np.mean(all_contributions[:, t]) for t in range(num_years)
    ]
    output_dict['Avg_Drawdown'] = [
        np.mean(all_drawdowns[:, t]) for t in range(num_years)
    ]
    output_dict['Avg_Real_Contribution'] = [
        np.mean(all_real_contributions[:, t]) for t in range(num_years)
    ]
    output_dict['Avg_Real_Drawdown'] = [
        np.mean(all_real_drawdowns[:, t]) for t in range(num_years)
    ]
    
    for i in range(10):
        output_dict[f"Alloc_Asset{i+1}"] = allocation_arrays[i]
    
    df_results = pd.DataFrame(output_dict)
    return df_results


In [19]:
num_sims = 1000
num_years = 80

# Suppose we have 10 assets, each with a time-varying mean & stdev
# For illustration, let's just do something simple:
#   Asset 1: "8" (8% constant), stdev "15" (15% constant)
#   Asset 2: "6 10S 5" means 6% first 9 yrs, step to 5% in year 10, stay at 5% ...
# etc. We'll keep it short here and fill the rest with constants:
asset_mean_vecs = [
    "8", "6", "5", "7", "9", "4", "3", "5 10S 7", "10", "2"
]
asset_vol_vecs = [
    "15", "12", "10", "13", "18", "8", "5", "10 10S 12", "20", "3"
]
asset_dist_types = ["normal"]*10  # all normal for now

# Suppose we have an allocation that goes from 80% equity / 20% fixed to 40% equity / 60% fixed over time
# We'll just do silly placeholders for the 10 assets. Let's say the first 5 assets are "equities",
# and the next 5 are "fixed income". We'll ramp from 15% each in the first 5 to 5% each by year 30,
# while the next 5 ramp from 1% each to 7% each, etc. Just an example:
# The sum doesn't necessarily have to be 100 unless you want it to. 
# We'll do each asset individually:
allocation_vecs = [
    "100",   # asset 1
    "0",   # asset 2
    "0",   # asset 3
    "0",   # asset 4
    "0",   # asset 5
    "0",    # asset 6
    "0",    # asset 7
    "0",    # asset 8
    "0",    # asset 9
    "0",    # asset 10
]

# Now call our function
df_results = monte_carlo_retirement(
    num_sims=num_sims,
    num_years=num_years,
    inflation_rate_vector="3",   
    inflation_vol_vector="0",                
    inflation_dist='normal',
    
    asset_mean_vectors=asset_mean_vecs,
    asset_vol_vectors=asset_vol_vecs,
    asset_dist_types=asset_dist_types,
    allocation_vectors=allocation_vecs,
    
    initial_contribution=10000.0,
    contrib_growth_vector="2 50S 0",   
    contrib_vol_vector="0",          
    contrib_dist='normal',
    peg_contribution_to_inflation=False,
    contribution_stop_year=50,
    
    initial_drawdown=80000,
    drawdown_growth_vector="0 50S 3",  
    drawdown_vol_vector="0",
    drawdown_dist='normal',
    peg_drawdown_to_inflation=False,
    drawdown_start_year=50, 
    
    initial_balance=0.0,
    random_seed=42
)
df_results.to_excel("output.xlsx")
df_results  


,Year,Balance_P10,Balance_P25,Balance_P50,Balance_P75,Balance_P90,Real_Balance_P10,Real_Balance_P25,Real_Balance_P50,Real_Balance_P75,...,Alloc_Asset1,Alloc_Asset2,Alloc_Asset3,Alloc_Asset4,Alloc_Asset5,Alloc_Asset6,Alloc_Asset7,Alloc_Asset8,Alloc_Asset9,Alloc_Asset10
0,1,8.969982e+03,9.802335e+03,1.077350e+04,1.185692e+04,1.252068e+04,8.708721e+03,9.516830e+03,1.045971e+04,1.151157e+04,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.823836e+04,2.026528e+04,2.250260e+04,2.504075e+04,2.709357e+04,1.719141e+04,1.910197e+04,2.121086e+04,2.360331e+04,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2.801998e+04,3.152518e+04,3.568881e+04,4.029502e+04,4.432828e+04,2.564225e+04,2.885001e+04,3.266032e+04,3.687565e+04,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,3.797018e+04,4.354463e+04,4.989870e+04,5.669806e+04,6.272963e+04,3.373601e+04,3.868884e+04,4.433435e+04,5.037549e+04,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,4.881201e+04,5.605642e+04,6.530811e+04,7.534696e+04,8.535750e+04,4.210567e+04,4.835476e+04,5.633535e+04,6.499495e+04,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,8.602904e+06,1.544475e+07,3.371554e+07,6.757913e+07,1.153344e+08,9.099468e+05,1.633623e+06,3.566162e+06,7.147983e+06,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,77,8.907636e+06,1.624952e+07,3.502281e+07,7.300043e+07,1.273310e+08,9.147368e+05,1.668685e+06,3.596539e+06,7.496510e+06,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,78,9.429182e+06,1.711561e+07,3.738154e+07,7.682015e+07,1.384738e+08,9.400923e+05,1.706431e+06,3.726950e+06,7.658992e+06,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,79,9.819035e+06,1.834908e+07,3.894340e+07,8.180152e+07,1.502156e+08,9.504474e+05,1.776125e+06,3.769581e+06,7.918093e+06,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
